## 1. Setup

In [ ]:
!pip install -q --upgrade transformers>=4.44.0 peft>=0.11.0 accelerate>=1.0.0 datasets>=2.20.0 \
    pillow>=10.4.0 opencv-python-headless>=4.10.0 jsonschema>=4.22.0 pymupdf>=1.24.9 pytesseract>=0.3.13 bitsandbytes
try:
    import bitsandbytes  # noqa: F401
except Exception as e:
    print("bitsandbytes unavailable, training will fall back to FP16", e)


## 2. Clone repo

In [ ]:
import os
if not os.path.exists('/content/fiche-idee-global-vlm'):
    !git clone https://github.com/placeholder/fiche-idee-global-vlm.git
%cd /content/fiche-idee-global-vlm
!pip install -q -r requirements.txt


## 3. Data check

In [ ]:
from pathlib import Path
from PIL import Image
vision_dir = Path('data/vision_crops')
images = sorted(vision_dir.glob('*.*'))
print(f'Found {len(images)} images')
for img_path in images[:2]:
    display(Image.open(img_path).resize((320, 320)))
if not images:
    print('Drop ~16 fiche crops into data/vision_crops/')


## 4. Build JSONL skeleton + annotate

In [ ]:
!python scripts/make_multiview_jsonl.py --images-dir data/vision_crops --output data/json_data/multiview_dataset.jsonl

from pathlib import Path
import json
from PIL import Image
dspath = Path('data/json_data/multiview_dataset.jsonl')
records = [json.loads(l) for l in dspath.read_text(encoding='utf-8').splitlines() if l.strip()]
print(f'Records ready: {len(records)}')

# Minimal inline annotator
idx = 0  # change index per fiche
rec = records[idx]
print(rec['images'])
for p in rec['images']:
    display(Image.open(p).resize((320,320)))
print('Current output:', json.dumps(rec['output'], indent=2, ensure_ascii=False))
# Edit below then save
rec['output'] = rec['output']  # <- replace with your filled JSON
records[idx] = rec
with dspath.open('w', encoding='utf-8') as f:
    for r in records:
        f.write(json.dumps(r, ensure_ascii=False) + '
')


## 5. Train LoRA

In [ ]:
!python training/train_qwen2vl_lora.py --jsonl data/json_data/multiview_dataset.jsonl --model Qwen/Qwen2-VL-2B-Instruct --output-dir outputs/qwen_lora_adapter


## 6. Test + export

In [ ]:
!python inference/predict_json.py --images data/vision_crops/sample.png --context "" --instruction default --adapter outputs/qwen_lora_adapter --output outputs/sample_pred.json
!python inference/json_to_openscad.py outputs/sample_pred.json --out outputs/sample.scad
from pathlib import Path
print(Path('outputs/sample_pred.json').read_text()[:1000])
